In [27]:
from VideoTools import VideoExtractor, VideoAnalyzer
from dotenv import load_dotenv, find_dotenv
import os
from openai import AzureOpenAI
from utils import check_and_reduce_image_size, display_moderation_results
import base64
from IPython.display import display, Markdown

In [2]:
load_dotenv(find_dotenv())

aoai_endpoint = os.getenv('AOAI_ENDPOINT')
aoai_key = os.getenv('AOAI_KEY')
gpt_deployment = os.getenv('GPT_DEPLOYMENT')

content_safety_endpoint = os.getenv('CONTENT_SAFETY_ENDPOINT')
content_safety_key = os.getenv('CONTENT_SAFETY_KEY')

aoai_client = AzureOpenAI(api_version="2024-05-01-preview", api_key=aoai_key, azure_endpoint=aoai_endpoint) 

analyzer = VideoAnalyzer(aoai_client, gpt_deployment, content_safety_endpoint, content_safety_key)


In [4]:
current_image_path = "images/image_0044.png"
content_safety_image_path = check_and_reduce_image_size(current_image_path)


In [7]:
with open(content_safety_image_path, "rb") as image_file:
    base64_encoded_string = base64.b64encode(image_file.read()).decode('utf-8')

In [8]:
cf_results = analyzer.content_safety_moderate_image(base64_encoded_string)

In [9]:
cf_results

{'categoriesAnalysis': [{'category': 'Hate', 'severity': 0},
  {'category': 'SelfHarm', 'severity': 0},
  {'category': 'Sexual', 'severity': 0},
  {'category': 'Violence', 'severity': 0}]}

In [32]:
hate_thresh_label = "safe"
selfharm_thresh_label = "med"
sexual_thresh_label = "med"
violence_thresh_label = "med"


severity_to_id = {"safe" : 0, "low" : 2, "med" : 4, "high" : 6}
id_to_severity = {0 : "safe", 2 : "low", 4 : "med", 6 : "high"}

severity_thresholds = {'Hate' : severity_to_id[hate_thresh_label],
                       'SelfHarm' : severity_to_id[selfharm_thresh_label],
                       'Sexual' : severity_to_id[sexual_thresh_label],
                       'Violence' : severity_to_id[violence_thresh_label]}

In [14]:
severity_thresholds

{'Hate': 4, 'SelfHarm': 4, 'Sexual': 4, 'Violence': 4}

In [33]:
cs_results_dict = {}

for item in cf_results['categoriesAnalysis']:
    print(item)
    cs_results_dict[item['category']] = {
        'filtered' : False if item['severity'] < severity_thresholds[item['category']] else True,
        'severity' : id_to_severity[item['severity']]}

cs_results_dict

{'category': 'Hate', 'severity': 0}
{'category': 'SelfHarm', 'severity': 0}
{'category': 'Sexual', 'severity': 0}
{'category': 'Violence', 'severity': 0}


{'Hate': {'filtered': True, 'severity': 'safe'},
 'SelfHarm': {'filtered': False, 'severity': 'safe'},
 'Sexual': {'filtered': False, 'severity': 'safe'},
 'Violence': {'filtered': False, 'severity': 'safe'}}

In [34]:
cs_results_markdown = display_moderation_results(cs_results_dict)
display(Markdown("**Image content safety**: " + cs_results_markdown))

**Image content safety**: Hate: <span style='color:red;'>safe</span>, SelfHarm: safe, Sexual: safe, Violence: safe